In [1]:
import pandas as pd
import torch

from run_no_val import run

from model_3d.Spherenet_NMR import SphereNet

from load_Glycosciencedb_3d import create_graph_experiment

##### 1, Load the Glycoscience dataset for 3D GNN models

This code is modified from the 2D loading method.

We first randomly split the glycans into training set and test set. 

Due to the overfitting and the size of the dataset, we do not include a validation set here 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
Create = create_graph_experiment()

train_data_exp, test_data_exp = Create.create_all_graph_list()

--------------------------loading NMR Graph List-------------------------------


100%|█████████████████████████████████████████| 299/299 [00:06<00:00, 45.54it/s]


##### 2, Initialize the node embedding size

The number of number of the embedding size, is the input to the model

In [4]:
node_embedding_size = train_data_exp[0].z.shape[1]

##### 3, Initialize and train the SphereNet,

Our implementation is modified from https://github.com/divelab/DIG/tree/dig-stable. 

To apply SphereNet to our tasks, we replaced the global pooling layer, which is needed for predicting the properties of whole molecules, and added a layer that maps the learned embedding of each atom to its NMR chemical shift.

In [5]:
model = SphereNet(energy_and_force=False, in_embed_size = node_embedding_size, cutoff=5.0, num_layers=2,
                  hidden_channels=128, out_channels=1, int_emb_size=64,
                  basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=128,
                  num_spherical=3, num_radial=6, envelope_exponent=5,
                  num_before_skip=1, num_after_skip=2, num_output_layers=1)
loss_func = torch.nn.L1Loss()
run3d = run()

In [ ]:
train_loss_list, test_rmse_list = run3d.run(device=device, train_dataset = train_data_exp, test_dataset = test_data_exp ,
                                            model = model, loss_func=loss_func,
                                            epochs=5, batch_size=2, vt_batch_size= 2, lr=0.001, lr_decay_factor=0.5, lr_decay_step_size=15)


#Params: 552934

=====Epoch 1

Training...


100%|█████████████████████████████████████████| 120/120 [00:02<00:00, 40.47it/s]



Testing...



100%|███████████████████████████████████████████| 30/30 [00:00<00:00, 73.12it/s]


{'Train': 8.396147856613, 'Test': 0.9185028}

=====Epoch 2

Training...


...


100%|█████████████████████████████████████████| 120/120 [00:02<00:00, 50.31it/s]



Testing...



100%|███████████████████████████████████████████| 30/30 [00:00<00:00, 74.02it/s]


{'Train': 0.37055806890130044, 'Test': 0.3965973}

=====Epoch 4

Training...



 83%|██████████████████████████████████▏      | 100/120 [00:02<00:00, 51.09it/s]